In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [14]:
#read in the data
path = '/media/popo/Elements/Datasets/Wine Reviews/'
df = pd.read_csv(f'{path}wine_data.csv')
print('Shape: ', df.shape)

Shape:  (129971, 14)


In [15]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [16]:
df['variety'].unique().shape

(708,)

In [17]:
#we will take only the top 5 varieties
top_varieties = df['variety'].value_counts().head(5).index
df_top_5 = df[df['variety'].isin(top_varieties)]
df_top_5.shape


(50358, 14)

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence

In [23]:
sequence.pad_sequences([[1,2,3], [4,5,6,7]], maxlen=5, padding='post', truncating='post')

array([[1, 2, 3, 0, 0],
       [4, 5, 6, 7, 0]], dtype=int32)